In [44]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException, WebDriverException
import pandas as pd
from connection import db_string
import sqlalchemy as sql
from sqlalchemy.orm import sessionmaker
import csv

# Webscraping

### Collecting list of links for all 'Modern' events in time range

In [106]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.mtggoldfish.com/tournament_searches/create?utf8=%E2%9C%93&tournament_search%5Bname%5D=&tournament_search%5Bformat%5D=modern&tournament_search%5Bdate_range%5D=01%2F01%2F2022+-+08%2F16%2F2022&commit=Search")
driver.implicitly_wait(5)

In [13]:
# for loop for to go through each event on a page

#closing ad that gets in way of click
try:
    ad = driver.find_element(By.TAG_NAME, 'polygon')
    ad.click()
except StaleElementReferenceException:
    pass
except NoSuchElementException: 
    pass
except ElementNotInteractableException:
    pass

#getting page number to loop through
pagination = driver.find_element(By.CLASS_NAME, 'pagination')
num_pages = int(pagination.find_element(By.XPATH, "//li[@class='page-item'][4]").text)

#list for event links
links = []

for i in range(14):
    #closing ad that gets in way of click
    try:
        ad = driver.find_element(By.TAG_NAME, 'polygon')
        ad.click()
    except StaleElementReferenceException:
        pass
    except NoSuchElementException: 
        pass
    except ElementNotInteractableException:
        pass
    
    #navigate to list of tournament events
    results = driver.find_element(By.TAG_NAME, "tbody")

    #get all events on page as list
    events = results.find_elements(By.TAG_NAME, 'tr')

    # loop through the links and save the links in a list
    for event in events:
        link = event.find_element(By.TAG_NAME, 'a').get_attribute('href')
        links.append(link)
    print(f"page {i+1}")
    driver.find_element(By.LINK_TEXT, '→').click()

print(links)

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
['https://www.mtggoldfish.com/tournament/40311', 'https://www.mtggoldfish.com/tournament/40306', 'https://www.mtggoldfish.com/tournament/40302', 'https://www.mtggoldfish.com/tournament/40304', 'https://www.mtggoldfish.com/tournament/40270', 'https://www.mtggoldfish.com/tournament/40293', 'https://www.mtggoldfish.com/tournament/40273', 'https://www.mtggoldfish.com/tournament/40276', 'https://www.mtggoldfish.com/tournament/40284', 'https://www.mtggoldfish.com/tournament/40265', 'https://www.mtggoldfish.com/tournament/40252', 'https://www.mtggoldfish.com/tournament/40249', 'https://www.mtggoldfish.com/tournament/40237', 'https://www.mtggoldfish.com/tournament/40245', 'https://www.mtggoldfish.com/tournament/40240', 'https://www.mtggoldfish.com/tournament/40233', 'https://www.mtggoldfish.com/tournament/40224', 'https://www.mtggoldfish.com/tournament/40232', 'https://www.mtggoldfish.com/tou

In [10]:
#now can begin looping through list of links to get decks

#for loop of links navigates to that event page
    #for loop to collect list of links for deck lists

#new for loop that goes through list of decklist links
    #decklist scraping for loop


### Collecting list of url links to every deck in each tournament

In [21]:
#creating variable for deck links
deck_links = []

#loop to go through all tournaments
for link in links:
    
    #loads tournament page
    driver.get(f'{link}')
    
    #saving deck elements as list
    table = driver.find_element(By.CLASS_NAME, 'table-tournament')
    decks = table.find_elements(By.XPATH, "//a[starts-with(@href, '/deck/') and not(contains(@href, 'visual')) and not(contains(@href, 'custom'))]")

    for deck in decks:
        deck_links.append(deck.get_attribute('href'))
    
    print('event done')



event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done

In [23]:
#saving links to csv as a backup
deck_links_df = pd.DataFrame(deck_links)
deck_links_df.to_csv('decklinks.csv')

In [26]:
length = len(deck_links)
length

11233

In [100]:
cards_df = pd.DataFrame(columns=('Card Name', 'Quantity', 'Date'))
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
save_state = []

#initializing variables to keep progress
length = len(deck_links)
i = 1

for link in deck_links:
    
    try:
        driver.get(f'{link}')
        driver.implicitly_wait(5)
        
    except WebDriverException:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.get(f'{link}')
        driver.implicitly_wait(5)
    #closing ad that gets in way of click
    try:
        ad = driver.find_element(By.TAG_NAME, 'polygon')
        ad.click()
    except StaleElementReferenceException:
        pass
    except NoSuchElementException: 
        pass
    except ElementNotInteractableException:
        pass

    #collecting the info of the deck and event
    
    info = driver.find_element(By.CLASS_NAME, "deck-container-information")
    #split to find Date: and then get the three items after that entry 
    
    string = info.find_element(By.XPATH, "//*[text()[contains(.,'Deck Date')]]").text
    strings = string.split()
    start = strings.index('Date:')
    
    #save those three as the date
    date_string = strings[start+1:start+4]
    date = date_string[0] +' '+ date_string[1] +" "+ date_string[2]
    print(date)
    
    #get a list of the cards 
    card_list = driver.find_element(By.CLASS_NAME, 'deck-view-deck-table')
    cards = card_list.find_elements(By.TAG_NAME, 'tr')
    
    for card in cards:

        if card.get_attribute('class') == 'deck-category-header':
            continue

        quant = card.find_element(By.CLASS_NAME, 'text-right').text

        name = card.find_element(By.CLASS_NAME, 'card_id').text

        data = {"Card Name": name, "Quantity": int(quant), "Date":date}

        card_data = pd.DataFrame(data, columns=("Card Name", "Quantity", "Date"), index=[0])

        cards_df= pd.concat([cards_df, card_data], ignore_index=True)

    #scraping of deck done
    #display progress
    print(f"Deck {i} of {length}")
    
    #saving checkpoints
    if i % 50 == 0:
        cards_df.to_csv('cards_df.csv')
        save_state=[]
        
        for link in deck_links[i+1:]:
            save_state.append(link)
            
        save_state_df = pd.DataFrame(save_state)
        save_state_df.to_csv('save_state.csv', index=False)
        print('Checkpoint saved')
        
    i+=1


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=104.0.5112.81)
Stacktrace:
Backtrace:
	Ordinal0 [0x009278B3+2193587]
	Ordinal0 [0x008C0681+1771137]
	Ordinal0 [0x007D41A8+803240]
	Ordinal0 [0x007BC910+706832]
	Ordinal0 [0x00820139+1114425]
	Ordinal0 [0x0082D282+1168002]
	Ordinal0 [0x0081D5C6+1103302]
	Ordinal0 [0x007F77E0+948192]
	Ordinal0 [0x007F86E6+952038]
	GetHandleVerifier [0x00BD0CB2+2738370]
	GetHandleVerifier [0x00BC21B8+2678216]
	GetHandleVerifier [0x009B17AA+512954]
	GetHandleVerifier [0x009B0856+509030]
	Ordinal0 [0x008C743B+1799227]
	Ordinal0 [0x008CBB68+1817448]
	Ordinal0 [0x008CBC55+1817685]
	Ordinal0 [0x008D5230+1856048]
	BaseThreadInitThunk [0x7533FA29+25]
	RtlGetAppContainerNamedObjectPath [0x771B7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x771B7A6E+238]


In [73]:
cards_df.to_csv('cards_df.csv')

In [69]:
### loading in list of links from save state

,Card Name,Quantity,Date
0,Memnite,4,"Aug 16, 2022"
1,Ornithopter,4,"Aug 16, 2022"
2,Esper Sentinel,4,"Aug 16, 2022"
3,Lion Sash,1,"Aug 16, 2022"
4,Puresteel Paladin,4,"Aug 16, 2022"
5,Stoneforge Mystic,4,"Aug 16, 2022"
6,Blacksmith's Skill,1,"Aug 16, 2022"
7,Steelshaper's Gift,3,"Aug 16, 2022"
8,Colossus Hammer,4,"Aug 16, 2022"
9,Shadowspear,1,"Aug 16, 2022"


In [74]:
cards_df

,Card Name,Quantity,Date
0,Memnite,4,"Aug 16, 2022"
1,Ornithopter,4,"Aug 16, 2022"
2,Esper Sentinel,4,"Aug 16, 2022"
3,Lion Sash,1,"Aug 16, 2022"
4,Puresteel Paladin,4,"Aug 16, 2022"
...,...,...,...
350987,Blood Moon,2,"Jan 14, 2022"
350988,Mystical Dispute,4,"Jan 14, 2022"
350989,Dead // Gone,2,"Jan 14, 2022"
350990,Memory Deluge,1,"Jan 14, 2022"


In [88]:
csv_file = pd.read_csv('save_state.csv', header= None, names = ['Index', 'Link'])
save_state_links = csv_file['Index'].tolist()

save_state_links.remove('0')


['https://www.mtggoldfish.com/deck/4554279',
 'https://www.mtggoldfish.com/deck/4554280',
 'https://www.mtggoldfish.com/deck/4554281',
 'https://www.mtggoldfish.com/deck/4554282',
 'https://www.mtggoldfish.com/deck/4554283',
 'https://www.mtggoldfish.com/deck/4554284',
 'https://www.mtggoldfish.com/deck/4554285',
 'https://www.mtggoldfish.com/deck/4552224',
 'https://www.mtggoldfish.com/deck/4552225',
 'https://www.mtggoldfish.com/deck/4552227',
 'https://www.mtggoldfish.com/deck/4552226',
 'https://www.mtggoldfish.com/deck/4552228',
 'https://www.mtggoldfish.com/deck/4549355',
 'https://www.mtggoldfish.com/deck/4549337',
 'https://www.mtggoldfish.com/deck/4549326',
 'https://www.mtggoldfish.com/deck/4549300',
 'https://www.mtggoldfish.com/deck/4549317',
 'https://www.mtggoldfish.com/deck/4549318',
 'https://www.mtggoldfish.com/deck/4549353',
 'https://www.mtggoldfish.com/deck/4549291',
 'https://www.mtggoldfish.com/deck/4549327',
 'https://www.mtggoldfish.com/deck/4549302',
 'https://

In [108]:
#crashed at xxx48 so database has entries from those 48 decks already. 
# Start at link 49 in the save state
save_state_links = save_state_links[50:]
len(save_state_links)

282

In [101]:
### loading in card dataframe from save state
cards_df = pd.read_csv('cards_df.csv')
cards_df

,Card Name,Quantity,Date
0,Memnite,4,16-Aug-22
1,Ornithopter,4,16-Aug-22
2,Esper Sentinel,4,16-Aug-22
3,Lion Sash,1,16-Aug-22
4,Puresteel Paladin,4,16-Aug-22
...,...,...,...
350987,Blood Moon,2,14-Jan-22
350988,Mystical Dispute,4,14-Jan-22
350989,Dead // Gone,2,14-Jan-22
350990,Memory Deluge,1,14-Jan-22


In [103]:
save_state_links

['https://www.mtggoldfish.com/deck/4554279',
 'https://www.mtggoldfish.com/deck/4554280',
 'https://www.mtggoldfish.com/deck/4554281',
 'https://www.mtggoldfish.com/deck/4554282',
 'https://www.mtggoldfish.com/deck/4554283',
 'https://www.mtggoldfish.com/deck/4554284',
 'https://www.mtggoldfish.com/deck/4554285',
 'https://www.mtggoldfish.com/deck/4552224',
 'https://www.mtggoldfish.com/deck/4552225',
 'https://www.mtggoldfish.com/deck/4552227',
 'https://www.mtggoldfish.com/deck/4552226',
 'https://www.mtggoldfish.com/deck/4552228',
 'https://www.mtggoldfish.com/deck/4549355',
 'https://www.mtggoldfish.com/deck/4549337',
 'https://www.mtggoldfish.com/deck/4549326',
 'https://www.mtggoldfish.com/deck/4549300',
 'https://www.mtggoldfish.com/deck/4549317',
 'https://www.mtggoldfish.com/deck/4549318',
 'https://www.mtggoldfish.com/deck/4549353',
 'https://www.mtggoldfish.com/deck/4549291',
 'https://www.mtggoldfish.com/deck/4549327',
 'https://www.mtggoldfish.com/deck/4549302',
 'https://

In [109]:

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
save_state = []

#initializing variables to keep progress
length = len(save_state_links)
i = 1

#crashed at xxx48 so database has entries from those 48 decks already. 
# Start at link 49 in the save state

for link in save_state_links:
    
    try:
        driver.get(f'{link}')
        
        driver.implicitly_wait(5)
        
    except WebDriverException:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.get(f'{link}')
        driver.implicitly_wait(5)
    #closing ad that gets in way of click
    try:
        ad = driver.find_element(By.TAG_NAME, 'polygon')
        ad.click()
    except StaleElementReferenceException:
        pass
    except NoSuchElementException: 
        pass
    except ElementNotInteractableException:
        pass

    #collecting the info of the deck and event
    
    info = driver.find_element(By.CLASS_NAME, "deck-container-information")
    #split to find Date: and then get the three items after that entry 
    
    string = info.find_element(By.XPATH, "//*[text()[contains(.,'Deck Date')]]").text
    strings = string.split()
    start = strings.index('Date:')
    
    #save those three as the date
    date_string = strings[start+1:start+4]
    date = date_string[0] +' '+ date_string[1] +" "+ date_string[2]
    print(date)
    
    #get a list of the cards 
    card_list = driver.find_element(By.CLASS_NAME, 'deck-view-deck-table')
    cards = card_list.find_elements(By.TAG_NAME, 'tr')
    
    for card in cards:

        if card.get_attribute('class') == 'deck-category-header':
            continue

        quant = card.find_element(By.CLASS_NAME, 'text-right').text

        name = card.find_element(By.CLASS_NAME, 'card_id').text

        data = {"Card Name": name, "Quantity": int(quant), "Date":date}

        card_data = pd.DataFrame(data, columns=("Card Name", "Quantity", "Date"), index=[0])

        cards_df= pd.concat([cards_df, card_data], ignore_index=True)

    #scraping of deck done
    #display progress
    print(f"Deck {i} of {length}")
    
    #saving checkpoints
    if i % 50 == 0:
        cards_df.to_csv('cards_df.csv')
        save_state=[]
        
        for link in save_state_links[i+1:]:
            save_state.append(link)
            
        save_state_df = pd.DataFrame(save_state)
        save_state_df.to_csv('save_state.csv', index=False)
        print('Checkpoint saved')
        
    i+=1

Jan 14, 2022
Deck 1 of 282
Jan 14, 2022
Deck 2 of 282
Jan 14, 2022
Deck 3 of 282
Jan 14, 2022
Deck 4 of 282
Jan 14, 2022
Deck 5 of 282
Jan 14, 2022
Deck 6 of 282
Jan 14, 2022
Deck 7 of 282
Jan 14, 2022
Deck 8 of 282
Jan 14, 2022
Deck 9 of 282
Jan 14, 2022
Deck 10 of 282
Jan 14, 2022
Deck 11 of 282
Jan 14, 2022
Deck 12 of 282
Jan 14, 2022
Deck 13 of 282
Jan 14, 2022
Deck 14 of 282
Jan 14, 2022
Deck 15 of 282
Jan 14, 2022
Deck 16 of 282
Jan 14, 2022
Deck 17 of 282
Jan 14, 2022
Deck 18 of 282
Jan 14, 2022
Deck 19 of 282
Jan 14, 2022
Deck 20 of 282
Jan 14, 2022
Deck 21 of 282
Jan 14, 2022
Deck 22 of 282
Jan 14, 2022
Deck 23 of 282
Jan 14, 2022
Deck 24 of 282
Jan 14, 2022
Deck 25 of 282
Jan 14, 2022
Deck 26 of 282
Jan 14, 2022
Deck 27 of 282
Jan 14, 2022
Deck 28 of 282
Jan 13, 2022
Deck 29 of 282
Jan 13, 2022
Deck 30 of 282
Jan 13, 2022
Deck 31 of 282
Jan 12, 2022
Deck 32 of 282
Jan 12, 2022
Deck 33 of 282
Jan 12, 2022
Deck 34 of 282
Jan 12, 2022
Deck 35 of 282
Jan 12, 2022
Deck 36 of 282
J

In [110]:
cards_df.to_csv('cards_df.csv')